# Trabalho Prático 2 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Criptosistemas pós-quânticos PKE/KEM* - Kyber/Crystalis

Implementar o esquema Kyber/Crystalis em classes *Python/SageMath* apresentando, para cada um, as versões **KEM-IND-CPA** e **PKE-IND-CCA**.

Nota: Baseado no documento Kyber/Crystalis (https://pq-crystals.org/kyber/data/kyber-specification-round3.pdf)

### Kyber/Crystalis - KEM-IND-CPA

In [302]:
import math, os
from hashlib import sha3_512 as G, shake_128 as XOF, sha3_256 as H, shake_256 as PRF

'''
Parâmetros definidos para a implementação do kyber
'''
n = 256
_n = 9
q = 7681
Qq = PolynomialRing(GF(q), 'x')
y = Qq.gen()
RQ = QuotientRing(Qq, y^n+1)

'''
Definição da função mod+-
'''
def modMm(r,a) :
    _r = r % a
    # Testar se a é par
    if mod(a,2)==0 :
        # Cálculo dos limites -a/2 e a/2
        inf_bound, sup_bound = -a/2, a/2
    # a é ímpar
    else :
        # Cálculo dos limites -a-1/2 e a-1/2
        inf_bound, sup_bound = (-a-1)/2, (a-1)/2
    # Queremos garantir que o módulo se encontre no intervalo calculado
    while _r > sup_bound :
        _r-=a
    while _r < inf_bound :
        _r+=a
    return _r
    
    

'''
Função que converte um array de bytes num array de bits
'''
def bytesToBits(bytearr) :
    bitarr = []
    for elem in bytearr :
        bitElemArr = []
        # Calculamos cada bit pertencente ao byte respetivo
        for i in range(0,8) :
            bitElemArr.append(mod(elem // 2**(mod(i,8)),2))
        
        for i in range(0,len(bitElemArr)) :
            bitarr.append(bitElemArr[i])
    return bitarr

'''
Função que converte um array de bits num array de bytes
'''
def bitsToBytes(bitarr) :
    bytearr = []
    bit_arr_size = len(bitarr)
    byte_arr_size = bit_arr_size / 8
    for i in range(byte_arr_size) :
        elem = 0
        for j in range(8) : # Definir macro BYTE_SIZE = 0
            elem += (int(bitarr[i*8+j]) * 2**j)
        bytearr.append(elem)
    return bytearr
    
        


''' 
Função parse cuja finalidade é receber como input 
uma byte stream e retornar, como output, a representação NTT
'''
def parse(b) :
    coefs = [0]*n # O poly terá n=256 coeficientes
    i,j = 0,0
    while j<n :
        d = b[i] + 256*b[i+1]
        d = mod(d,2**13)
        if d<q :
            coefs[br(8,j)] = d
            j+=1
        i+=2
    return RQ(coefs)

'''
Implementação da função que implementa 
o bit reversed order

Parâmetros:
    - _bits : nº de bits usados para representar nr
    - nr : valor a ser bitreversed
'''
def br(_bits, nr) :
    res = 0
    for i in range(_bits) :
        res += (nr % 2) * 2**(_bits-i-1)
        nr = nr // 2
    return res
    

'''
Definição da função compress
'''
def compress(q,x,d) :
    return mod(round((2**d)/q * x),2**d)

'''
Definição da função decompress
'''
def decompress(q,x,d) :
    return round((q/2**d) * x)


'''
Definição da função CBD. Recebe como 
input o n (comprido) e o array de bytes
'''
def cbd(noise, btarray) :
    f = []
    bitArray = bytesToBits(btarray)
    for i in range(256) :
        a, b = 0, 0
        # Cálculo do a e do b
        for j in range(256) :
            a+=bitArray[2*i*noise + j]
            b+=bitArray[2*i*noise + noise + j]
        f.append(a - b)
    return RQ(f)


'''
Implementação da função decode
'''
def decode(l, btarray) :
    f = []
    bitArray = bytesToBits(btarray)
    for i in range(256) :
        fi = 0
        for j in range(l) :
            fi += int(bitArray[i*l+j]) * 2**j
        f.append(fi)
    return RQ(f)

'''
Implementação da função encode
'''
def encode(l, poly) :
    bitArr = []
    coef_array = poly.list()
    # Percorremos cada coeficiente
    for i in range(256) :
        actual = int(coef_array[i])
        for j in range(l) :
            bitArr.append(actual % 2)
            actual = actual // 2
    return bitsToBytes(bitArr)

'''
Implementação da função NTT
'''
def ntt(g) :
    
    w, psi = 3844, 62 # Parâmetros definidos no documento da especificação do kyber
    
    '''
    Função auxiliar que calcula cada um dos coeficientes de NTT(g).
    
    Parâmetros:
        - g : elemento para o qual vai ser calculado o coeficiente
        - i : indice o coeficiente de g que va i ser calculado
    '''
    def g_i(g, i) :
        g_i = 0
        for j in range(n) :
            g_i += psi**i * g[j] * w**(i*j)
        return g_i
    
    coefs = [0] * n
    for i in range(n) :
        coefs[i] = g_i(g,i)
        
    return RQ(coefs)

'''
Implementação da função NTT-1
'''
def ntt_inv(_g) :
    
    w, psi = 3844, 62 # Parâmetros definidos no documento da especificação do kyber
    
    '''
    Função auxiliar que calcula cada um dos coeficientes de g.
    
    Parâmetros:
        - g : elemento para o qual vai ser calculado o coeficiente
        - i : indice o coeficiente de g que va i ser calculado
    '''
    def _g_i(_g, i) :
        g_i = n**-1 * psi**-1
        soma = 0
        for j in range(n) :
            soma += int(_g[j]) * w**(-i*j)
        return int(g_i)*int(soma)
    
    coefs = [0] * n
    for i in range(n) :
        print('A calcular coeficiente g^', i)
        coefs[i] = _g_i(_g,i)
        
    return RQ(coefs)

'''
Função que retorna um vetor resultante da 
multiplicação entre uma matriz M e um vetor v
'''
def multMatrixVector(M,v,k) :
    T = NTT()
    As = []
    for i in range(k-1) :
        As.append(0)
        for j in range(k-1) :
            As[i] += T.ntt_inv(M[i][j] * v[j])
    return As
            


In [286]:
arr = [32,42,34,5,35,3,5,7,54,34,21,43,5,2,46,7,3,3,21,43,53,0,0,0,0,0,0,0,0,0,0,0,32,42,34,5,35,3,5,7,54,34,21,43,5,2,46,7,3,3,21,43,53,0,0,0,0,0,0,0,0,0,0,0]
poly = decode(2,arr)
print(len(poly.list()))
print(poly.list())
#print(len(encode(2,poly)))
print(arr)
print(encode(2,poly))
if arr == encode(2,poly) :
    print('\n[Success] Os arrays correspondem!')
else :
    print('\n[Erro] Algo correu mal com o decode/encode!')

256
[0, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0, 1, 1, 0, 0, 3, 0, 2, 0, 3, 0, 0, 0, 1, 1, 0, 0, 3, 1, 0, 0, 2, 1, 3, 0, 2, 0, 2, 0, 1, 1, 1, 0, 3, 2, 2, 0, 1, 1, 0, 0, 2, 0, 0, 0, 2, 3, 2, 0, 3, 1, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 1, 1, 1, 0, 3, 2, 2, 0, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0, 1, 1, 0, 0, 3, 0, 2, 0, 3, 0, 0, 0, 1, 1, 0, 0, 3, 1, 0, 0, 2, 1, 3, 0, 2, 0, 2, 0, 1, 1, 1, 0, 3, 2, 2, 0, 1, 1, 0, 0, 2, 0, 0, 0, 2, 3, 2, 0, 3, 1, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 1, 1, 1, 0, 3, 2, 2, 0, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 5, 2, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 5, 2, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[32, 42, 3

In [154]:
# Teste para as funções NTT e NTT-1
F = GF(q) ;  R = PolynomialRing(F, name="w")
f = R.random_element(64)
print('A calcular ntt...')
ff = ntt(f)
print('A calcular ntt-1...')
fff = ntt_inv(ff)
if(f == fff) :
    print('[Success]')
else :
    print('[Error] Something was wrong')

A calcular ntt...
A calcular ntt-1...
A calcular coeficiente g^ 0
A calcular coeficiente g^ 1
A calcular coeficiente g^ 2
A calcular coeficiente g^ 3
A calcular coeficiente g^ 4
A calcular coeficiente g^ 5
A calcular coeficiente g^ 6
A calcular coeficiente g^ 7
A calcular coeficiente g^ 8
A calcular coeficiente g^ 9
A calcular coeficiente g^ 10
A calcular coeficiente g^ 11
A calcular coeficiente g^ 12


KeyboardInterrupt: 

In [235]:
'''
Implementação da classe NTT
'''
class NTT(object):
#    
    def __init__(self, n=256):
        if not any([n == t for t in [32,64,128,256,512,1024,2048]]):
            raise ValueError("improper argument ",n)
        self.n = n  
        self.q = 2*n+1
        while True:
            if (self.q).is_prime():
                break
            self.q += 2*n
        print(self.q)
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="xbar")
        w = (self.R).gen(); self.w = w
        
        g = (w^n + 1)
        print(len(g.roots(multiplicities=False)))
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])  
    
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def ntt_inv(self,ff):                              ## transformada inversa
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)

In [236]:
# Teste

T = NTT()

f = T.random_pol(255)
#print(f)

ff = T.ntt(f)
#print(ff)

fff = T.ntt_inv(ff)

# print(fff)
print("Correto ? ",f == fff)

7681
256
Correto ?  True


In [303]:
'''
Implementação da classe 
'''

class KyberPKE :
    
    def __init__(self,n,k,q,nn,du,dv,dt) :
        
        self.n = n
        self.k = k
        self.q = q
        self.nn = nn
        self.du = du
        self.dv = dv
        self.dt = dt
        Qq = PolynomialRing(GF(q), 'x')
        y = Qq.gen()
        RQ = QuotientRing(Qq, y^n+1)
        self.Rq = RQ
        
    '''
    Função que permite a geração de 
    uma chave pública
    '''
    def keygen(self, sk) :
        d = os.urandom(32)
        h = G(d)
        digest = h.digest()
        ro,sigma = digest[:256], digest[256:]
        N = 0
        A, s, e = [], [], []
        for i in range(self.k-1) :
            A.append([])
            for j in range(self.k-1) :
                xof = XOF()
                xof.update(ro + j.to_bytes(4,'little') + i.to_bytes(4,'little'))
                A[i].append(parse(xof.digest(int(q))))
        for i in range(self.k-1) :
            prf = PRF()
            prf.update(sigma + int(N).to_bytes(4,'little'))
            s.append(cbd(self.nn, prf.digest(int(q+1))))
            N += 1
        for i in range(self.k-1) :
            prf = PRF()
            prf.update(sigma + int(N).to_bytes(4,'little'))
            e.append(cbd(self.nn, prf.digest(int(q))))
            N += 1
        T = NTT()
        _s = [T.ntt(s[0])]
        
        As = multMatrixVector(A,_s,self.k)

In [301]:
k = KyberPKE(256,2,3329,3,2,10,4)
k.keygen(os.urandom(768))

7681
256
7681
256
xbar^253 + xbar^252 + xbar^251 + xbar^249 + xbar^248 + xbar^247 + xbar^246 + xbar^245 + xbar^244 + xbar^242 + xbar^241 + xbar^240 + xbar^238 + xbar^237 + xbar^236 + xbar^234 + xbar^233 + xbar^232 + xbar^231 + xbar^230 + xbar^229 + xbar^228 + xbar^226 + xbar^225 + xbar^224 + xbar^223 + xbar^222 + xbar^220 + xbar^218 + xbar^216 + xbar^214 + xbar^213 + xbar^210 + xbar^205 + xbar^204 + xbar^203 + xbar^202 + xbar^201 + xbar^197 + xbar^196 + xbar^193 + xbar^189 + xbar^187 + xbar^186 + xbar^185 + xbar^184 + xbar^183 + xbar^177 + xbar^174 + xbar^173 + xbar^172 + xbar^171 + xbar^170 + xbar^169 + xbar^166 + xbar^163 + xbar^162 + xbar^161 + xbar^158 + xbar^156 + xbar^155 + xbar^151 + xbar^150 + xbar^149 + xbar^148 + xbar^147 + xbar^146 + xbar^144 + xbar^143 + xbar^142 + xbar^140 + xbar^135 + xbar^134 + xbar^132 + xbar^131 + xbar^125 + xbar^121 + xbar^117 + xbar^116 + xbar^115 + xbar^110 + xbar^109 + xbar^106 + xbar^105 + xbar^104 + xbar^102 + xbar^98 + xbar^97 + xbar^96 + xbar^8

TypeError: can't multiply sequence by non-int of type 'list'

In [5]:
br(8,113)

142

In [89]:
i,j = 1,1
xof = XOF()
xof.update(os.urandom(256) + int(j).to_bytes(4,'little') + int(i).to_bytes(4,'little'))
digest = xof.digest(int(q))

f = parse(digest)

T = NTT()

ff = T.ntt(f)

fff = T.ntt_inv(ff)

# print(fff)
print("Correto ? ",f == fff)

256
Correto ?  False
